Bu notebook eğitilmiş RNN modellerini kolay kullanılabilir bir kullanıcı arayüzüne aktarmak için oluşturulmuştur.
Diğer modeller için olan Gradio notebook'larının da çoğu açıklaması buradadır.

# Setup

In [1]:
import gradio as gr
import torch
import torch.nn as nn
import numpy as np
import json

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# PHASE 1: Get the model

Bu kısımda modellerin mimarisi tanımlanıp -asıl eğitildikleri notebook'takinin aynısı- kaydedilen modeller notebook'a yüklenir

In [2]:
# Modeli tanımla
class BiGRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.0):
        super(BiGRUModel, self).__init__()
        self.bi_gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)
        self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        out, _ = self.bi_gru(x)
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        
        return out

In [3]:
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.0):
        super(BiLSTMModel, self).__init__()
        self.bi_lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        out, _ = self.bi_lstm(x)
        out = self.fc(out[:, -1, :])
        
        return out

In [4]:
# Modeli yükle
model_GRU = torch.load(fr"MODELS\main_GRU_model.pth")
model_GRU.eval()
model_GRU

BiGRUModel(
  (bi_gru): GRU(3, 128, num_layers=2, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [5]:
model_LSTM = torch.load(fr"MODELS\main_LSTM_model.pth")
model_LSTM.eval()
model_LSTM

BiLSTMModel(
  (bi_lstm): LSTM(3, 64, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

# PHASE 2: Configure the IU

Üç tane ana kullanıcı ara yüzü tanımladım:
1) Single Prediction: Girilen verilere ve seçilen ayarlara göre tek bir zaman noktası tahmini yapar
2) Forecast Plot: Girilen verilere ve seçilen ayarlara göre istenilen aralığı öngörür ve grafiğini çıkartır
3) Forecast Table: Girilen verilere ve seçilen ayarlara göre istenilen aralığı öngörür ve tablosunu çıkartır

Her arayüz için kullanıcın ne yapması gerektiğini iyice anlatmaya çalıştım, arayüzün olduğu bölümü çalıştırırsanız açıklamaları daha net görebilirsiniz.

In [6]:
# Denormalizasyon istatistikleri
with open(r"normalization_stats.json", "r") as json_file:
    norm_dict = json.load(json_file)

mean = norm_dict['mean']
std = norm_dict['std']

In [7]:
# Kullanıcının cevabını istenilen değişkene dönüştürmek için sözlük
dayofweek = {
    "Pazartesi": 1,
    "Salı": 2,
    "Çarşamba": 3,
    "Perşembe": 4,
    "Cuma": 5,
    "Cumartesi": 6,
    "Pazar": 7
}

## Predict Single

### Prediction logic

In [8]:
def predict(model_name, hour_interval, day_of_week, value_before):
    ## Modele verilecek girdiyi ayarlama kısmı
    value_before_norm = (value_before - mean) / std
    sequence = np.array([hour_interval, dayofweek[day_of_week], value_before_norm])
    torch_input = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    
    ## Seçilen modele göre:
    if model_name == "GRU":
        output = model_GRU(torch_input)
    elif model_name == "LSTM":
        output = model_LSTM(torch_input)

    ## Çıktıyı denormalize edip ekrana yansıt
    output_denorm = output * std + mean
    
    return round(output_denorm.item(), 2)

### UI

In [9]:
single_article = "**Nasıl Kullanılır**:\n * İstediğiniz modeli seçin.\n" \
            "* Tahmin etmek istediğiniz saati girin, günü seçin ve bir saat öncesindeki değeri sağlayın.\n" \
            "* 'Submit' düğmesine tıklayın. Tahmin edilen değer bölümü istenen sonucu gösterecektir. "

In [10]:
prediction_interface = gr.Interface( 
    fn=predict,
    inputs=[
        gr.Dropdown(choices=["GRU", "LSTM"], label="Model türü", value="GRU"),
        gr.Number(label="Saat Aralığı", value=14),
        gr.Dropdown(choices= ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"], label="Haftanın Günü", value="Cuma"),
        gr.Number(label="Bir önceki değer", value=30000)
    ],
    outputs=gr.Textbox(label="Tahmin edilen değer"),
    title="Öngörü Uygulaması",
    description="Model Türü, Saat Aralığı, Haftanın Günü ve Önceki Değer'i girerek bir tahmin alın.",
    theme=gr.themes.Soft(),
    article=single_article,
    allow_flagging="never"
)

prediction_interface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


## Forecast Plot

### Forecasting Logic

In [11]:
desired_column = "NetAmountWOVat"
features = ["HourInterval", "DayOfWeek", f'{desired_column} Before']

def forecast_plot(model_name, start_timestamp, end_timestamp, sequence_length=1):
    forecast_timestamps = pd.date_range(start=start_timestamp, end=end_timestamp, freq='H')
    forecast_df = pd.DataFrame(index=forecast_timestamps)
    
    forecast_df[f'Forecasted {desired_column}'] = np.nan
    forecast_df[f'{desired_column} Before'] = np.nan
    forecast_df["HourInterval"] = forecast_df.index.hour
    forecast_df['DayOfWeek'] = forecast_df.index.dayofweek + 1
    
    forecast_df = forecast_df[[f'Forecasted {desired_column}', 'HourInterval', 'DayOfWeek', f'{desired_column} Before']]
    
    forecast_df.iloc[:sequence_length + 1, forecast_df.columns.get_loc(f'{desired_column} Before')] = (0-mean)/std
    
    if model_name == "GRU":
        for i in range(0, len(forecast_df) + 100):
            sequence = forecast_df.iloc[i:i + sequence_length]
            sequence = sequence[features].values
            
            torch_input = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)

            predicted_value = model_GRU(torch_input)

            forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')] = predicted_value.item()
            if forecast_df.isnull().sum()[0] == 0:
                break

            forecast_df.iloc[i + 1, forecast_df.columns.get_loc(f'{desired_column} Before')] = forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')]
    
    elif model_name == "LSTM":
        for i in range(0, len(forecast_df) + 100):
            sequence = forecast_df.iloc[i:i + sequence_length]
            sequence = sequence[features].values
            
            torch_input = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)

            predicted_value = model_LSTM(torch_input)

            forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')] = predicted_value.item()
            if forecast_df.isnull().sum()[0] == 0:
                break

            forecast_df.iloc[i + 1, forecast_df.columns.get_loc(f'{desired_column} Before')] = forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')]

    forecast_df_denorm = forecast_df.copy()
    forecast_df_denorm[f'Forecasted {desired_column}'] = forecast_df[f'Forecasted {desired_column}'] * std + mean
    
    # Grafiği yansıt
    return px.line(forecast_df_denorm, x=forecast_df.index, y=f'Forecasted {desired_column}')

### UI

In [12]:
plot_article = "Nasıl Kullanılır:\n * İstediğiniz modeli seçin.\n" \
                "* Başlangıç ve bitiş tarihini YY-AA-GG SS:DD:SS biçiminde girin ve geçmişe bakma aralığını sağlayın. \n"\
                "* 'Submit' düğmesine tıklayın. Tahminler bir grafik olarak gösterilecektir. "

In [13]:
plot_interface = gr.Interface(
    fn=forecast_plot,
    inputs=[
        gr.Dropdown(choices=["GRU", "LSTM"], label="Model Türü", value="GRU"),
        gr.Textbox(label="Başlangıç Zamanı (YYYY-AA-GG SS:DD:SS)", value="2024-02-8 00:00:00"),  
        gr.Textbox(label="Bitiş Zamanı (YYYY-AA-GG SS:DD:SS)", value="2024-02-15 23:00:00")
      ],
    outputs="plot",
    title="Tahmin Uygulaması",
    description="Tahmin yapmak için başlangıç ve bitiş zamanını YYYY-AA-GG SS:DD:SS formatında girin.",
    article = plot_article,
    allow_flagging="never"
)

plot_interface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


## Forecast Table

### Forecasting Logic

In [14]:
desired_column = "NetAmountWOVat"
features = ["HourInterval", "DayOfWeek", f'{desired_column} Before']

def forecast_table(model_name, start_timestamp, end_timestamp, sequence_length=1):
    forecast_timestamps = pd.date_range(start=start_timestamp, end=end_timestamp, freq='H')
    forecast_df = pd.DataFrame(index=forecast_timestamps)
    
    forecast_df[f'Forecasted {desired_column}'] = np.nan
    forecast_df[f'{desired_column} Before'] = np.nan
    forecast_df["HourInterval"] = forecast_df.index.hour
    forecast_df['DayOfWeek'] = forecast_df.index.dayofweek + 1
    
    forecast_df = forecast_df[[f'Forecasted {desired_column}', 'HourInterval', 'DayOfWeek', f'{desired_column} Before']]
    
    forecast_df.iloc[:sequence_length + 1, forecast_df.columns.get_loc(f'{desired_column} Before')] = (0-mean)/std

    if model_name == "GRU":
        for i in range(0, len(forecast_df) + 100):
            sequence = forecast_df.iloc[i:i + sequence_length]
            sequence = sequence[features].values
            
            torch_input = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)

            predicted_value = model_GRU(torch_input)

            forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')] = predicted_value.item()
            if forecast_df.isnull().sum()[0] == 0:
                break

            forecast_df.iloc[i + 1, forecast_df.columns.get_loc(f'{desired_column} Before')] = forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')]
    
    elif model_name == "LSTM":
        for i in range(0, len(forecast_df) + 100):
            sequence = forecast_df.iloc[i:i + sequence_length]
            sequence = sequence[features].values
            
            torch_input = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)

            predicted_value = model_LSTM(torch_input)

            forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')] = predicted_value.item()
            if forecast_df.isnull().sum()[0] == 0:
                break

            forecast_df.iloc[i + 1, forecast_df.columns.get_loc(f'{desired_column} Before')] = forecast_df.iloc[i, forecast_df.columns.get_loc(f'Forecasted {desired_column}')]

    forecast_df_denorm = forecast_df.copy()
    forecast_df_denorm[f'Forecasted {desired_column}'] = forecast_df[f'Forecasted {desired_column}'] * std + mean

    formatted_timestamps = forecast_df_denorm.index.strftime("%Y-%m-%d %H:%M:%S").tolist()
    predicted_values = forecast_df_denorm[f'Forecasted {desired_column}'].tolist() 

    data = list(zip(formatted_timestamps, predicted_values))

    return gr.Dataframe(value=data, headers=["Time", f"Predicted {desired_column}"])

### UI

In [15]:
table_article = "Nasıl Kullanılır:\n * İstediğiniz modeli seçin.\n" \
                "* Başlangıç ve bitiş tarihini YY-AA-GG SS:DD:SS biçiminde girin ve geçmişe bakma aralığını sağlayın. \n"\
                "* 'Submit' düğmesine tıklayın. Tahminler size tablo halinde gösterilecektir. "

In [16]:
interface = gr.Interface(
    fn=forecast_table,
    inputs=[
        gr.Dropdown(choices=["GRU", "LSTM"], label="Model Türü", value="GRU"),
        gr.Textbox(label="Başlangıç Zamanı (YYYY-AA-GG SS:DD:SS)", value="2024-02-8 00:00:00"),  
        gr.Textbox(label="Bitiş Zamanı (YYYY-AA-GG SS:DD:SS)", value="2024-02-10 23:00:00")
      ],
    outputs=gr.Dataframe(label="Tahmin Tablosu"),
    title="Tahmin Uygulaması",
    description="Tahmin yapmak için başlangıç ve bitiş zamanını YYYY-AA-GG SS:DD:SS formatında girin.",
    article=table_article,
    allow_flagging="never"
)

interface.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
